### 编辑距离
- 优点：可解释性强，在论文判重中就可以知道哪里增删改查
- 缺点：对句子很长时效率低，两个句子的相似性不涉及逻辑问题

### 单词的表征
##### One-hot 表征
   - 如果单词很长，占空间大
   - 利用率不高，稀疏矩阵的情况
   
##### PCA方法 
    将多维降下来的方法，降维过程会产生一定损失，在单词表征中，用一个单词矩阵表示，如果有新的语料，需要整个矩阵的重新计算
#### 好的单词的表征方法
   - 节省空间
   - 可调整更新，（adaptive update）
   - 可表征语义相似性
   
**单词为何表征为向量**：用于计算

## Part1:  Word2Vec

    gensim的输入只要求序列化的句子，而不需要将所有输入都存储在内存中，简单说，可以输入一个句子，处理，删除，再载入新的
    例如分散在硬盘的多个文件，每个句子一行，不需要将所有的输入先行存储在内存中，word2vec可以一个一个文件一行一行进行处理
    
    调用 Word2Vec(sentences, iter=1) 会调用句子迭代器运行两次（一般来说，会运行 iter+1 次，默认情况下  iter=5）。第一次运行负责收集单词和它们的出现频率，从而构造一个内部字典树。第二次以及以后的运行负责训练神经模型。
    
   ### 训练
    Word2vec 有多个影响训练速度和质量的参数。
   - **min_count** :
       
     其中之一是用来修剪内部字典树的。在一个数以亿计的预料中出现一到两次的单词非常有可能是噪音或不需要被关注的。另外，也没有足够的数据对他们进行有意义的训练。因此，最好的办法就是直接将他们忽略掉。
    <br><code>model = Word2Vec(sentences, min_count=10) # default value is 5</code>
     <br>对于设定 min_count 的值，合理的范围是0 - 100，可以根据数据集的规模进行调整。
    
   - **size** 
    
    另一个参数是神经网络 NN 层单元数，它也对应了训练算法的自由程度。
   <br><code>model = Word2Vec(sentences, size=200) # default value is 100</code>
     <br>更大的 size 值需要更多的训练数据，但也同时可以得到更准确的模型。合理的取值范围是几十到几百。
   - **workers** 
        
    最后一个主要参数是训练并行粒度，用来加速训练。
   <br><code>model = Word2Vec(sentences, workers=4) # default = 1 worker = no parallelization</code>
        <br>该参数只有在机器已安装 Cython 情况下才会起到作用。如没有 Cython，则只能单核运行。


In [2]:
from gensim.models.Word2Vec import LineSentence

In [7]:
from gensim.models import Word2Vec

In [3]:
senetences = [['cat','say','meow'],['dog','say','woof']]

In [8]:
model = Word2Vec(senetences, min_count=1)

In [9]:
model.wv.vocab

{'cat': <gensim.models.keyedvectors.Vocab at 0x204986646d8>,
 'say': <gensim.models.keyedvectors.Vocab at 0x20498664710>,
 'meow': <gensim.models.keyedvectors.Vocab at 0x20498664748>,
 'dog': <gensim.models.keyedvectors.Vocab at 0x20498664780>,
 'woof': <gensim.models.keyedvectors.Vocab at 0x204986647b8>}

In [10]:
model.most_similar('cat')

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woof', 0.11478932201862335),
 ('dog', 0.1006193608045578),
 ('meow', 0.05883640795946121),
 ('say', -0.07562532275915146)]

In [12]:
database = 'F:/Jupyter/NLP/Artificial-Intelligence-NLP/Project1/sqlResult_1558435.csv'

In [13]:
import pandas as pd

In [43]:
content = pd.read_csv(database)

In [51]:
samples = content['content'].tolist()

In [32]:
import jieba

In [69]:
def cut(string): return ' '.join(list(jieba.cut(string)))

In [76]:
import re
def token(content):
    return "".join(re.findall('[\w|\d]+', content))

In [77]:
cut(token(samples[1]))

'骁龙 835 作为 唯一 通过 Windows10 桌面 平台 认证 的 ARM 处理器 高通 强调 不会 因为 只 考虑 性能 而 去 屏蔽掉 小 核心 相反 他们 正 联手 微软 找到 一种 适合 桌面 平台 的 兼顾 性能 和 功耗 的 完美 方案 报道 称 微软 已经 拿到 了 一些 新 的 源码 以便 Windows10 更好 地 理解 biglittle 架构 资料 显示 骁龙 835 作为 一款 集成 了 CPUGPU 基带 蓝牙 WiFi 的 SoC 比 传统 的 Wintel 方案 可以 节省 至少 30 的 PCB 空间 按计划 今年 Q4 华硕 惠普 联想 将 首发 骁龙 835Win10 电脑 预计 均 是 二合一 形态 的 产品 当然 高通 骁龙 只是 个 开始 未来 也许 还 能 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 等 进入 Windows10 桌面 平台'

### Try a larger corpus(新闻语料库)

In [79]:
with open('corpus_samples.txt','w', encoding='utf-8') as f:
    for s in samples:
        f.write(cut(token(str(s))) + '\n')

In [80]:
sentences = LineSentence('corpus_samples.txt')  # 传入的可以是文件，自动一行一行处理，但是序列化的，分好词的

In [81]:
model = Word2Vec(sentences, min_count=1)

In [83]:
model.wv['小米']

array([-0.13104406,  0.38645142, -0.07911395, -0.6025887 , -1.4590093 ,
       -0.641312  ,  0.23608142,  0.44704896, -0.17430197,  0.5703797 ,
       -0.7349662 ,  0.6174384 , -0.19381727,  0.4355138 , -0.14908443,
        0.08480924,  0.1478671 ,  0.65789557, -1.6371346 , -1.6961427 ,
       -0.6820333 , -0.76239157,  0.11705615,  0.60525763, -1.3686242 ,
       -0.641274  ,  0.4820115 , -0.5392673 ,  0.6774031 , -0.8671957 ,
        0.41286898,  0.15779582,  0.49609366, -0.25388467, -0.6672813 ,
        1.1358349 ,  0.49193388,  0.4392241 ,  0.2954627 , -1.6753192 ,
        0.05976949, -0.35856354, -1.1684122 , -1.3162006 , -0.91744447,
        1.1383    ,  0.01991438,  0.14978853,  0.94313556, -0.1938932 ,
       -0.24554539,  0.927059  , -0.84104836,  0.8830491 , -0.3140727 ,
        0.45615208,  0.1681259 ,  0.53862   , -0.5258455 , -0.69867676,
       -0.7801806 ,  1.0568571 ,  0.25144058,  1.4197085 ,  1.0986097 ,
        0.5151397 ,  1.3575599 ,  0.5901367 , -0.35268053,  1.01

In [82]:
model.most_similar('小米')

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('苹果', 0.7384345531463623),
 ('华为', 0.7256077527999878),
 ('天猫', 0.7211840748786926),
 ('格力', 0.7203738689422607),
 ('三星', 0.714936375617981),
 ('商城', 0.7115495800971985),
 ('京东', 0.7076677680015564),
 ('家电', 0.7018177509307861),
 ('沃尔玛', 0.6988679766654968),
 ('亚马逊', 0.6985962390899658)]

In [84]:
model.most_similar('华为')

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('三星', 0.8133171796798706),
 ('谷歌', 0.7662756443023682),
 ('英特尔', 0.7578772306442261),
 ('微软', 0.7469288110733032),
 ('旗下', 0.7316062450408936),
 ('IT', 0.7305045127868652),
 ('阿里巴巴', 0.7290788888931274),
 ('西门子', 0.7272427082061768),
 ('小米', 0.7256077527999878),
 ('中国电信', 0.7227927446365356)]

## Part2 : Name Entity Recognition

     区分出什么是人名，机构组织名，地点

In [91]:
from jieba import posseg

     jieba.posseg 标注分词后每个词的词性，采用ictclas的标记法，对于此行注释
     https://blog.csdn.net/suibianshen2012/article/details/53487157

In [94]:

text = """新华社华盛顿4月26日电 美国总统特朗普26日表示，美国将撤销在《武器贸易条约》上的签字。

特朗普当天在美国印第安纳州首府印第安纳波利斯举行的美国全国步枪协会年会上说，《武器贸易条约》是一个“严重误导的条约”，美国将撤销在该条约上的签字，联合国将很快收到美国正式拒绝该条约的通知。"""

In [95]:
words = posseg.cut(text)

In [99]:
for w in posseg.cut(text):
    print(w)

新华社/nt
华盛顿/ns
4/m
月/m
26/m
日电/j
 /x
美国/ns
总统/n
特朗普/nr
26/m
日/m
表示/v
，/x
美国/ns
将/d
撤销/v
在/p
《/x
武器/n
贸易/vn
条约/n
》/x
上/f
的/uj
签字/v
。/x

/x

/x
特朗普/nr
当天/t
在/p
美国/ns
印第安纳州/ns
首府/n
印第安纳波利斯/ns
举行/v
的/uj
美国/ns
全国/n
步枪/n
协会/n
年/m
会上/t
说/v
，/x
《/x
武器/n
贸易/vn
条约/n
》/x
是/v
一个/m
“/x
严重/a
误导/n
的/uj
条约/n
”/x
，/x
美国/ns
将/d
撤销/v
在/p
该/r
条约/n
上/f
的/uj
签字/v
，/x
联合国/nt
将/d
很快/d
收到/v
美国/ns
正式/ad
拒绝/v
该/r
条约/n
的/uj
通知/v
。/x
